In [1]:
import os
from fairseq2.models.llama import LLaMABuilder,LLaMAConfig,LLaMATokenizer,LLaMAConfig,llama_archs
from fairseq2.data import VocabularyInfo
from transformers import AutoTokenizer,AutoConfig,AutoModelForCausalLM
import torch
import json
from pathlib import Path
from fairseq2.models.sequence import SequenceBatch
from fairseq2.nn.padding import PaddingMask,pad_seqs,apply_padding_mask
import torch.nn.functional as F
import random
from fairseq2.tasks.generation import GenerationTask,CodellamaGenerationTask
from fairseq2.tasks.generation.generation import generation_tasks

/data/aigc/miniconda3/envs/ncc/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
task = GenerationTask(task_name="codellama_7b_code",device="cuda:2")

In [3]:
task.set_tokenizer(LLaMATokenizer('/data/lz/models/CodeLlama-7b/tokenizer.model'))
task.load_state('/data/lz/models/CodeLlama-7b')
task.load_dataset('/data/lz/ncc2/src/dataset.json')

In [ ]:
task.run(output_path="/data/lz/ncc2/src/result.json",batch_size=1,max_length=50,temperature=0.5,top_p=0.99)

In [4]:
task.generate(['this is','def main'],max_length=10,top_k=20,top_p=0.99,penalty_weight=0.8)[0]

/data/lz/ncc2/src/fairseq2/tasks/generation/generation.py:113: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sqs_token_ids = torch.tensor(sqs_token_ids,dtype=torch.int64).to(self.device)


'this is a b c\n  a.'

In [ ]:
# 1. generation sample fix (add beam search,etc.)
# 2. task register fix √
# 3. hf model adapter
# 4. code completion task